In [1]:
!pip install transformers

In [2]:
!pip install simpletransformers

In [3]:
!pip install torch

In [4]:
import json
import random
import warnings
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import torch
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adagrad
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, f1_score
from sklearn.utils import class_weight
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer

In [5]:
df = pd.read_excel(open('data.xlsx', 'rb'), dtype={'text': str, 'label': int})  

In [6]:
df.loc[df['label'] == 1, 'category'] = 'positive'
df.loc[df['label'] == -1, 'category'] = 'negtive'
df['label'] = df['label'].replace(-1,0)

In [7]:
#data split for train and test
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

In [8]:
from simpletransformers.classification import ClassificationModel

In [9]:
#for bert text column should be string and label column should be int
train["text"]=train["text"].apply(lambda r: str(r))
train['label']=train['label'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
#create model with turkish bert
model = ClassificationModel('bert', 'dbmdz/bert-base-turkish-uncased', num_labels=2,
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 2, "train_batch_size": 32 , "fp16":True, "output_dir": "bert_model"},use_cuda=True)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-turkish-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

In [11]:
train.head()

,text,label,category
17573,ı nvşif i̇ngilter de malvarlıklarınj satıp kar...,1,positive
10322,bahçeliev bançeııevıer patla ol bina ger harab...,1,positive
21135,maltep den tunceli̇ ye bin metre cem yapt mâ m...,1,positive
930,selams uç ev yer eş burak öztürk selams uç ev ...,1,positive
10842,yayinci i̇smai̇l ceyha si türki̇ye de i̇kli̇m ...,1,positive


In [12]:
#train model
model.train_model(train)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:449: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/17106 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/535 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/535 [00:00<?, ?it/s]

(1070, 0.19156240982136244)

In [13]:
result, model_outputs, wrong_predictions = model.eval_model(test)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/4277 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/535 [00:00<?, ?it/s]

In [14]:
predictions = model_outputs.argmax(axis=1)
actuals = test.label.values

In [15]:
from sklearn import metrics
print(metrics.classification_report(actuals, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.911     0.941     0.926      1382
           1      0.972     0.956     0.964      2895

    accuracy                          0.951      4277
   macro avg      0.941     0.949     0.945      4277
weighted avg      0.952     0.951     0.952      4277



In [16]:
from google.colab import drive
drive.mount('/content/drive')

bert_model  cache_dir  data.xlsx  runs	sample_data


In [ ]:
%cd /content/drive/My Drive/BERT SAVE

In [25]:
import pickle
filename = 'bert_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [26]:
filename = 'bert_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))


In [27]:
result, model_outputs, wrong_predictions = loaded_model.eval_model(test)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/4277 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/535 [00:00<?, ?it/s]

In [28]:
predictions = model_outputs.argmax(axis=1)
actuals = test.label.values

In [29]:
from sklearn import metrics
print(metrics.classification_report(actuals, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.911     0.941     0.926      1382
           1      0.972     0.956     0.964      2895

    accuracy                          0.951      4277
   macro avg      0.941     0.949     0.945      4277
weighted avg      0.952     0.951     0.952      4277



In [41]:
test


,text,label,category
11170,dev ihanet e devlet mi besleyecek cumhurbaşka ...,0,negtive
13197,seferberli̇k i̇la etti̇k beyok beledi taraf ko...,1,positive
14763,istiklal mars yarışmaç kazana bell olt bu yıl ...,1,positive
4667,q ğjfcğ istanbul kül ve turizm bakanlık nın ka...,1,positive
20862,i̇hale ayni adre if hkum birlik i̇nşaat ihal a...,0,negtive
...,...,...,...
8915,cemmoruı uher yarişiyor çekmeköy beledi nin çe...,1,positive
19294,ı bahçeliev de sık tedbir türki̇ye de de koron...,1,positive
16419,sokak sokak duaşarak knsbveroip horoz şeker da...,1,positive
2541,tank da dost gel gör yol aç fikret özka doğrut...,0,negtive


In [45]:
import io
data_string = """text, label,  category
                 bu gerçekten iyi,?,? """

data = io.StringIO(data_string)
df_temp = pd.read_csv(data, sep=",")

In [46]:
df_temp

,text,label,category
0,bu gerçekten iyi,?,?


In [47]:
loaded_model.eval_model(df_temp)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/1 [00:00<?, ?it/s]

ValueError: ignored

In [64]:
x = loaded_model.predict('TANIŞTIĞIMA MEMNUN OLDUM')
print(x)


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

(array([1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 0]), array([[-1.11718750e+000,  6.82128906e-001],
       [ 2.14321575e-312,  6.79038654e-313],
       [ 2.01589600e-312,  2.33419537e-312],
       [ 2.46151512e-312,  2.01589600e-312],
       [ 2.44029516e-312,  2.29175545e-312],
       [ 9.33678149e-313,  2.12199579e-312],
       [ 2.46151512e-312,  9.33678149e-313],
       [ 2.46151512e-312,  2.27053550e-312],
       [ 2.33419537e-312,  2.58883487e-312],
       [ 2.41907520e-312,  6.79038653e-313],
       [ 2.41907520e-312,  2.44029516e-312],
       [ 6.79038653e-313,  2.35541533e-312],
       [ 2.14321575e-312,  6.79038653e-313],
       [ 2.48273508e-312,  2.46151512e-312],
       [ 2.01589600e-312,  2.05833592e-312],
       [ 2.14321575e-312,  9.33678149e-313],
       [ 2.35541533e-312,  2.46151512e-312],
       [ 2.48273508e-312,  2.01589600e-312],
       [ 2.35541533e-312,  2.14321575e-312],
       [ 6.79038653e-313,  2.35541533e-312],
       [ 2.